# 라이브러리

In [1]:
import os
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import default_data_collator, DataCollatorForLanguageModeling, AutoTokenizer
from datasets import load_from_disk
from datasets import DatasetDict
from transformers import AutoConfig
from torch.optim import AdamW  # ✅ 이걸로 교체


import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from torch import nn
from datasets import load_dataset
import random
import re

/home/woogoonkyu/anaconda3/envs/lora_null/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터 셋 구성

- 1) saltlux/Ko-Llama3-Luxia-8B
- 2) beomi/Llama-3-KoEn-8B-Instruct-preview 
- 3) beomi/Llama-3-Open-Ko-8B
- 4) MLP-KTLim/llama-3-Korean-Bllossom-8B

In [ ]:
model_name1 = 'saltlux/Ko-Llama3-Luxia-8B'
model_name2 = 'beomi/Llama-3-KoEn-8B-Instruct-preview'
model_name3 = 'beomi/Llama-3-Open-Ko-8B'
model_name4 = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

In [3]:
tokenizer1 = AutoTokenizer.from_pretrained(model_name1)
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
tokenizer3 = AutoTokenizer.from_pretrained(model_name3)
tokenizer4 = AutoTokenizer.from_pretrained(model_name4)

# Modifying Prompt

In [4]:
def transform_prompt(prompt: str) -> str:
    # 추가할 지침 문장
    role_instruction = "You will always respond as the doctor, and your answer must be written in Korean only.\n\n"

    # 1. '***Conversation:' 위치 찾기
    parts = prompt.split('***Conversation:')
    if len(parts) != 2:
        return prompt  # 변환할 수 없는 구조는 그대로 반환

    header, conversation = parts

    # 2. Client:, Doctor: 라벨 제거
    conversation_cleaned = re.sub(r'^(Client|Doctor):\s*', '', conversation, flags=re.MULTILINE)

    # 3. 역할 문장을 Conversation 위에 삽입
    transformed_prompt = header.strip() + "\n\n" + role_instruction + "***Conversation:" + conversation_cleaned

    return transformed_prompt

# 예시
example = """You are a medical professional proficient in both Korean and English.
Your task is to carefully read the conversation between a Client and a Doctor, 
then predict the next response from the Doctor in a professional and empathetic manner.

Please respond in **Korean** only.

Now produce the next Doctor response to the Client’s last question.
Return your answer in Korean.

***Conversation:
Doctor: 알겠습니다. 그리고 소변이 느리게 흐르고, 물이 새어나오는 문제는 어떤 상황에서 더 심해지나요?
Client: 특별한 상황은 없고, 그냥 일상적으로 느린 것 같아요.

***Answer:"""

# 변환 실행
transformed = transform_prompt(example)
print(transformed)


You are a medical professional proficient in both Korean and English.
Your task is to carefully read the conversation between a Client and a Doctor, 
then predict the next response from the Doctor in a professional and empathetic manner.

Please respond in **Korean** only.

Now produce the next Doctor response to the Client’s last question.
Return your answer in Korean.

You will always respond as the doctor, and your answer must be written in Korean only.

***Conversation:
알겠습니다. 그리고 소변이 느리게 흐르고, 물이 새어나오는 문제는 어떤 상황에서 더 심해지나요?
특별한 상황은 없고, 그냥 일상적으로 느린 것 같아요.

***Answer:


In [5]:
dataset = load_from_disk('combined_dataset_short')
def truncate_after_answer(example):
    if example['source'] == 'dialogue':
        example['input'] = transform_prompt(example['input'])
        example['output'] = re.sub(r'^(Client|Doctor):\s*', '', example['output'], flags=re.MULTILINE)

    if example['source'] == 'qa':
        example['input'] = example['input'].replace('Choice','선택')
        example['input'] = example['input'].replace('Answer:','***Answer:').replace('Question:','***Question:' )
        example['input'] = example['input'].strip()
        example['output'] = example['output'].replace('Choice','선택').strip()
        
    if '***Answer:' in example['input']:
        # '***Answer:'까지만 남기고 뒷부분 삭제
        idx = example['input'].index('***Answer:') + len('***Answer:')
        example['input'] = example['input'][:idx].strip()
    return example

# 적용
train_cleaned = dataset['train'].map(truncate_after_answer)
test_cleaned = dataset['test'].map(truncate_after_answer)

# 다시 DatasetDict로
cleaned_dataset = DatasetDict({
    'train': train_cleaned,
    'test': test_cleaned
})


cleaned_dataset.save_to_disk("cleaned_dataset_with_answer_tag_new")

Saving the dataset (1/1 shards): 100%|██████████| 3870/3870 [00:00<00:00, 281966.34 examples/s]


In [6]:
cleaned_dataset = load_from_disk('cleaned_dataset_with_answer_tag_new')

In [7]:

cleaned_dataset['train']['source'][0], cleaned_dataset['test']['source'][-1]

('dialogue', 'qa')

In [8]:
cleaned_dataset['test']['source'][0], cleaned_dataset['test']['source'][-1]

('dialogue', 'qa')

In [9]:
print(cleaned_dataset['train']['input'][0])
print ('#' * 50)
print(cleaned_dataset['train']['input'][1])

You are a medical professional proficient in both Korean and English.
Your task is to carefully read the conversation between a Client and a Doctor, 
then predict the next response from the Doctor in a professional and empathetic manner.

Please respond in **Korean** only.

Now produce the next Doctor response to the Client’s last question.
Return your answer in Korean.

You will always respond as the doctor, and your answer must be written in Korean only.

***Conversation:
간전문의를 받으려면 어떻게 해야 하나요?
저희 병원에서 간전문의를 받을 수 있도록 예약을 도와드리겠습니다. 또한, 음주 중단과 혈당 관리에 대한 교육도 제공해드릴 수 있습니다.
감사합니다. 얼마나 빨리 예약을 잡아야 하나요?

***Answer:
##################################################
You are a medical professional proficient in both Korean and English.
Your task is to carefully read the conversation between a Client and a Doctor, 
then predict the next response from the Doctor in a professional and empathetic manner.

Please respond in **Korean** only.

Now produce the next Doctor response to the Client’s last

In [10]:
print(cleaned_dataset['train']['output'][0])
print ('#' * 50)
print(cleaned_dataset['train']['output'][1])

가능한 빨리 예약을 잡아드리겠습니다. 환자님의 상태를 고려하여 적절한 시기에 예약이 이루어질 수 있도록 도와드리겠습니다.
##################################################
혹시 배뇨 중에 통증이나 불편함을 느끼시나요?


In [11]:
print(cleaned_dataset['train']['input'][0])
print ('#' * 50)
print(cleaned_dataset['train']['input'][1])

You are a medical professional proficient in both Korean and English.
Your task is to carefully read the conversation between a Client and a Doctor, 
then predict the next response from the Doctor in a professional and empathetic manner.

Please respond in **Korean** only.

Now produce the next Doctor response to the Client’s last question.
Return your answer in Korean.

You will always respond as the doctor, and your answer must be written in Korean only.

***Conversation:
간전문의를 받으려면 어떻게 해야 하나요?
저희 병원에서 간전문의를 받을 수 있도록 예약을 도와드리겠습니다. 또한, 음주 중단과 혈당 관리에 대한 교육도 제공해드릴 수 있습니다.
감사합니다. 얼마나 빨리 예약을 잡아야 하나요?

***Answer:
##################################################
You are a medical professional proficient in both Korean and English.
Your task is to carefully read the conversation between a Client and a Doctor, 
then predict the next response from the Doctor in a professional and empathetic manner.

Please respond in **Korean** only.

Now produce the next Doctor response to the Client’s last

In [12]:
print(cleaned_dataset['test']['output'][-1])
print ('#' * 50)
print(cleaned_dataset['test']['output'][-2])

선택 5, 환자의 병력조사 후 급성 증상을 완화해준다.
##################################################
선택 3, 졸레드론산(zoledronic acid)


In [13]:
print(cleaned_dataset['test']['input'][-1])
print ('#' * 50)
print(cleaned_dataset['test']['input'][-2])

You are a medical professional proficient in both Korean and English. 
Your task is to carefully analyze a given medical question and its five answer choices, 
then provide the correct answer.
The answer choice will always follow the fixed format of 선택 1, 선택 2, 선택 3, 선택 4, 선택 5.
Return the response in the following format:
선택 answer

Now, analyze the following medical question and return the answer.

***Question: "갑자기 잇몸 전체가 아파요."라며 내원한 20세 여자에 대한 임상소견은 다음과 같다.
심한 구취
불량한 구강위생
자발적 치은출혈
변연치은에 분화구 형성
치간유두의 괴사성 형태
가장 먼저 할 처치는?
선택 1: 마취 후 생검을 시행한다.
선택 2: 치은연상 치석제거를 시행한다.
선택 3: 치은연하 육아조직제거를 시행한다.
선택 4: 치면세마 후 구강위생상태를 평가한다.
선택 5: 환자의 병력조사 후 급성 증상을 완화해준다.
***Answer:
##################################################
You are a medical professional proficient in both Korean and English. 
Your task is to carefully analyze a given medical question and its five answer choices, 
then provide the correct answer.
The answer choice will always follow the fixed format of 선택 1, 선택 2, 선택 3, 선택 4, 선택 5.
Re

# Preprocessing QA & Generation Dataset

In [14]:
from datasets import load_from_disk

dataset = load_from_disk('/nas_homes/projects/rapa/Dataset/data_updated_512_instruction')

In [15]:
dataset['test']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 8327
})

In [16]:
from transformers import AutoTokenizer
import pandas as pd

# 예시 tokenizer (사용 중인 모델에 맞게 수정하세요)
tokenizer = AutoTokenizer.from_pretrained("beomi/Llama-3-KoEn-8B-Instruct-preview")  # 또는 llama2, mistral 등

In [17]:
def decode_batch(example):
    input_ids = example["input_ids"]
    label_ids = [id for id in example["labels"] if id != -100]  # <- 여기 핵심!

    decoded_input = tokenizer.decode(input_ids, skip_special_tokens=True)
    decoded_label = tokenizer.decode(label_ids, skip_special_tokens=True)

    return {"decoded_input": decoded_input, "decoded_label": decoded_label}

decoded_dataset = dataset['test'].map(decode_batch)
df = decoded_dataset.to_pandas()[["decoded_input", "decoded_label"]]
df.rename(columns={"decoded_input": "input_text", "decoded_label": "label_text"}, inplace=True)

# 결과 확인
print(df.head())

Map: 100%|██████████| 8327/8327 [00:27<00:00, 303.05 examples/s]


                                          input_text  \
0  Please read the following content and respond ...   
1  Please read the following content and respond ...   
2  Please read the following content and respond ...   
3  Please read the following content and respond ...   
4  Please read the following content and respond ...   

                                          label_text  
0  환자의 징후와 증상, 부검 시 대뇌 피질에 루이체 소견이 있는 것으로 보아 루이체 ...  
1  당뇨병은 당뇨병성 신증으로 알려진 과정을 통해 만성 신장 질환으로 이어질 수 있습니...  
2  Title: 당뇨병 관리 - 새로 진단받은 환자를 위한 안내서\n\n소개:\n당뇨병...  
3  내분비계는 수많은 신체 기능을 조절하는 화학적 메신저인 호르몬을 생산하고 분비하는 ...  
4  폐암을 유발하는 위험 요인을 줄이려면 다음 단계를 수행할 수 있습니다:\n\n1. ...  


In [ ]:
# df.to_csv('COT_data.csv', encoding = 'utf-8-sig')

# Tokenizing Data with Mode Tokenizer

In [20]:
from datasets import load_from_disk
import torch
from transformers import AutoTokenizer
from datasets import DatasetDict
import random

# 모델에 맞는 tokenizer 로딩



tokenizer = tokenizer1

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 데이터셋 로딩
dataset = load_from_disk("cleaned_dataset_with_answer_tag_new")

def preprocess_function(example):
    prompt = example["input"]
    answer = example["output"]

    # "CLM 방식"에 맞게 전체 시퀀스를 하나로 연결
    full_text = prompt + " " + answer
    answer_tag = "***Answer:"

    # label을 마스킹하기 위한 기준점 위치
    answer_start = prompt.find(answer_tag)
    if answer_start == -1:
        if "Answer:" in prompt:
            prompt = prompt.replace('Answer:', "***Answer:")
        else:
            print(prompt)
            raise ValueError("prompt 안에 ***Answer: 태그가 없음")
        

    # full_text 전체를 토크나이즈
    tokenized = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors=None,
    )

    # label을 생성할 때, answer 시작 전까지는 -100
    prompt_tokenized = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        return_tensors=None,
    )

    tokenizer.pad_token_id = tokenizer.eos_token_id 

    labels = tokenized["input_ids"]
    masked_labels = [
        label if idx >= len(prompt_tokenized["input_ids"]) and label != tokenizer.pad_token_id else -100
        for idx, label in enumerate(labels)
    ]

    return {
        "input_ids": torch.tensor(tokenized["input_ids"]),
        "attention_mask": torch.tensor(tokenized["attention_mask"]),
        "labels": torch.tensor(masked_labels),
    }

In [21]:
tokenizer3.name_or_path.split('/')[0]

'beomi'

In [22]:
tokenizer1.name_or_path, tokenizer2.name_or_path, tokenizer3.name_or_path, tokenizer4.name_or_path

('saltlux/Ko-Llama3-Luxia-8B',
 'beomi/Llama-3-KoEn-8B-Instruct-preview',
 'beomi/Llama-3-Open-Ko-8B',
 'MLP-KTLim/llama-3-Korean-Bllossom-8B')

In [26]:
from tqdm import tqdm

num = 0
name_lst = []


model_name2 = 'beomi/Llama-3-KoEn-8B-Instruct-preview'
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)


for i in tqdm([tokenizer2]):

    tokenizer = i

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    name = tokenizer.name_or_path.split('/')[0].replace('-','_')

    # 데이터셋 전체에 전처리 적용
    tokenized_datasets = dataset.map(preprocess_function, batched=False)

    # 원본 텍스트 열 제거
    tokenized_datasets = tokenized_datasets.remove_columns(["input", "output", "source"])

    # 디스크에 저장
    if name in name_lst:
        name = name + str(num)
        num += 1
    else:
        pass
    tokenized_datasets.save_to_disk(f"tokenizer_dataset_shorts_{name}")
    name_lst.append(name)



  0%|          | 0/1 [00:00<?, ?it/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|██████████| 1/1 [00:35<00:00, 35.71s/it]


In [28]:
for i in tqdm([tokenizer2]):

    print('#################################################')
    tokenizer = i
    name = tokenizer.name_or_path.split('/')[0].replace('-','_')
    print(f'Next name : {name}')
    


    dataset = load_from_disk(f"tokenizer_dataset_shorts_{name}")

    num = 3240

    sample = dataset['train']['input_ids'][num]
    sample_output = [i for i in dataset['train']['labels'][num] if i != -100]
    print(tokenizer.decode(sample, skip_special_tokens= True))

    print(tokenizer.decode(sample_output)) # PAD, EOS -100으로 다 설정했으므로 Special token 안나와야함



  0%|          | 0/1 [00:00<?, ?it/s]

#################################################
Next name : beomi


100%|██████████| 1/1 [00:04<00:00,  4.67s/it]

You are a medical professional proficient in both Korean and English.
Your task is to carefully read the conversation between a Client and a Doctor, 
then predict the next response from the Doctor in a professional and empathetic manner.

Please respond in **Korean** only.

Now produce the next Doctor response to the Client’s last question.
Return your answer in Korean.

You will always respond as the doctor, and your answer must be written in Korean only.

***Conversation:
그럼 어떤 검사를 받아야 할까요? 그리고 어떻게 대처해야 할까요?
Chrystal 씨, 현재 증상과 과거 병력을 고려하여 복부 초음파 검사와 혈액 검사를 권장합니다. 이를 통해 감염의 정도와 기타 가능한 원인을 확인할 수 있습니다. 대처 방법은 항생제 치료와 함께 적절한 통증 완화 조치를 취하는 것입니다. 또한, 현재 당뇨병을 가지고 계신데, 혈당 관리에도 신경써야 합니다.
이해했습니다. 복부 초음파 검사와 혈액 검사를 받아보고 항생제와 통증 완화 조치를 취하겠습니다. 혈당 관리 역시 신경쓰겠습니다. 감사합니다.

***Answer: 별 말씀을요, Chrystal 씨. 건강에 더욱 신경 쓰시고 복원되시기를 기원합니다. 궁금한 점이 있으시면 언제든지 물어보세요.
 별 말씀을요, Chrystal 씨. 건강에 더욱 신경 쓰시고 복원되시기를 기원합니다. 궁금한 점이 있으시면 언제든지 물어보세요.


# 확인

In [30]:
name = ['saltlux','beomi_Koen_instruct','beomi0_Ko','MLP_KTLim']


model_name2 = 'beomi/Llama-3-KoEn-8B-Instruct-preview'
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)




dataset = load_from_disk(f"tokenizer_dataset_shorts_beomi")

num = -1

sample = dataset['test']['input_ids'][num]
sample_output = [i for i in dataset['test']['labels'][num] if i != -100]
print(tokenizer.decode(sample, skip_special_tokens= True))

print(tokenizer.decode(sample_output)) # PAD, EOS -100으로 다 설정했으므로 Special token 안나와야함



You are a medical professional proficient in both Korean and English. 
Your task is to carefully analyze a given medical question and its five answer choices, 
then provide the correct answer.
The answer choice will always follow the fixed format of 선택 1, 선택 2, 선택 3, 선택 4, 선택 5.
Return the response in the following format:
선택 answer

Now, analyze the following medical question and return the answer.

***Question: "갑자기 잇몸 전체가 아파요."라며 내원한 20세 여자에 대한 임상소견은 다음과 같다.
심한 구취
불량한 구강위생
자발적 치은출혈
변연치은에 분화구 형성
치간유두의 괴사성 형태
가장 먼저 할 처치는?
선택 1: 마취 후 생검을 시행한다.
선택 2: 치은연상 치석제거를 시행한다.
선택 3: 치은연하 육아조직제거를 시행한다.
선택 4: 치면세마 후 구강위생상태를 평가한다.
선택 5: 환자의 병력조사 후 급성 증상을 완화해준다.
***Answer: 선택 5, 환자의 병력조사 후 급성 증상을 완화해준다.
 선택 5, 환자의 병력조사 후 급성 증상을 완화해준다.
